In [2]:
from ultralytics import YOLO

In [ ]:
model = YOLO("yolov8n.pt")

In [ ]:
model.save('../models/yolov8n.pt')

In [ ]:
import torch

In [ ]:
model_torch = torch.load('../models/yolov8n.pt')

In [ ]:
num_layers = len(list(model.modules()))
print(f"Total number of layers/modules: {num_layers}")

In [ ]:
from torchinfo import summary

In [ ]:
summary(model.model, input_size=(1, 3, 640, 640))

In [ ]:
model


In [ ]:
freeze = [f"model.{x}." for x in range(10)]

In [ ]:
for k, v in model.named_parameters():
    v.requires_grad = True  # train all layers
    if any(x in k for x in freeze):
        print(f"freezing {k}")
        v.requires_grad = False

In [ ]:
for k , v in model.named_parameters():
    print(k, v.requires_grad)

In [ ]:
def train_model(model_path, data_path, save_path, stage, epochs=100, imgsz=640, batch_size=16, time = 44, run_dir='runs'):
    """
    Train or fine-tune a YOLO model.

    Parameters:
    - model_path (str): Path to the model file.
    - data_path (str): Path to the data configuration file.
    - save_path (str): Path to save the trained model.
    - stage (str): Training stage, either 'pretrain' or 'finetune'.
    - epochs (int): Number of training epochs. Default is 100.
    - imgsz (int): Image size for training. Default is 640.
    - batch_size (int): Batch size for training. Default is 16.
    """

    # Load the YOLO model
    model = YOLO(model_path)

    # Define training parameters
    train_params = {
        'data': data_path,
        'epochs': epochs,
        'imgsz': imgsz,
        'batch': batch_size,
        'device': 'cuda' if torch.cuda.is_available() else 'cpu',  # Use GPU if available
        'plots': True,
        'time': time,
        'patience': 100,
        'project': run_dir  # Specify the directory for storing runs
    }

    if stage == 'pretrain':
        # Start pre-training
        print("Starting pre-training...")
        results = model.train(**train_params)
        # Save the pre-trained model
        model.save(save_path)
    elif stage == 'finetune':
        # Start fine-tuning
        print("Starting fine-tuning...")
        results = model.train(**train_params)
        # Save the fine-tuned model
        model.save(save_path)

    # Print training results
    print(results)

# Pre-training stage:
# Uncomment the block below for the pre-training stage
# if __name__ == "__main__":
#     stage = 'pretrain'  # Specify 'pretrain' for the pre-training stage
#     train_model(
#         model_path='Models/yolov8n.pt',  # Path to the initial YOLO model
#         data_path='data.yaml',  # Path to the data configuration file for pre-training
#         save_path='Models/pre_trained_yolov8s.pt',  # Path to save the pre-trained model
#         stage=stage,  # Specify the stage: 'pretrain' or 'finetune'
#         epochs=100  # Number of training epochs for pre-training
#     )

# Fine-tuning stage:


In [3]:
import supervision as sv

In [4]:
COLORS = ['#FF1493', '#00BFFF', '#FF6347', '#FFD700']

In [5]:
BOX_ANNOTATOR = sv.BoxAnnotator(
    color=sv.ColorPalette.from_hex(COLORS),
    thickness=2
)

In [6]:
BOX_LABEL_ANNOTATOR = sv.LabelAnnotator(
    color=sv.ColorPalette.from_hex(COLORS),
    text_color=sv.Color.from_hex('#FFFFFF'),
    text_padding=5,
    text_thickness=1,
)

In [7]:
from typing import Iterator, List

In [8]:
import numpy as np

In [9]:
import cv2

## Player detection

In [26]:
def run_player_detection(source_video_path: str) -> Iterator[np.ndarray]:
    """
    Run player detection on a video and yield annotated frames.

    Args:
        source_video_path (str): Path to the source video.

    Yields:
        Iterator[np.ndarray]: Iterator over annotated frames.
    """
    player_detection_model = YOLO("../models/yolov8n_transfer_road_model.pt").to(device="cuda")
    frame_generator = sv.get_video_frames_generator(source_path=source_video_path)
    for frame in frame_generator:
        result = player_detection_model(frame, imgsz=1280, verbose=False)[0]
        detections = sv.Detections.from_ultralytics(result)

        annotated_frame = frame.copy()
        annotated_frame = BOX_ANNOTATOR.annotate(annotated_frame, detections)
        annotated_frame = BOX_LABEL_ANNOTATOR.annotate(annotated_frame, detections)
        yield annotated_frame

In [27]:
frame_generator = run_player_detection(
            source_video_path="../test_videos/test_video2.mp4")

In [28]:
video_info = sv.VideoInfo.from_video_path("../test_videos/test_video2.mp4")
with sv.VideoSink("result", video_info) as sink:
    for frame in frame_generator:
        sink.write_frame(frame)

        cv2.imshow("frame", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
        
    cv2.destroyAllWindows()

[ WARN:0@1122.334] global cap.cpp:643 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.10.0) /io/opencv/modules/videoio/src/cap_images.cpp:430: error: (-215:Assertion failed) !filename_pattern.empty() in function 'open'




In [31]:
import cv2
import numpy as np
from typing import Iterator
from ultralytics import YOLO

def run_player_detection(source_video_path: str) -> Iterator[np.ndarray]:
    """
    Run player detection on a video and yield annotated frames.

    Args:
        source_video_path (str): Path to the source video.

    Yields:
        Iterator[np.ndarray]: Iterator over annotated frames.
    """
    # Load YOLO model
    player_detection_model = YOLO("../models/yolov8n_transfer_based_model.pt").to(device="cuda")
    
    # Open the video file using OpenCV
    cap = cv2.VideoCapture(source_video_path)
    
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (1280, 960))
        
        # Run detection on the current frame
        result = player_detection_model(frame, imgsz=1280, verbose=False)[0]
        detections = sv.Detections.from_ultralytics(result)

        # Annotate the frame
        annotated_frame = frame.copy()
        annotated_frame = BOX_ANNOTATOR.annotate(annotated_frame, detections)
        annotated_frame = BOX_LABEL_ANNOTATOR.annotate(annotated_frame, detections)
        
        # Yield the annotated frame
        yield annotated_frame

    cap.release()

# Example usage of the generator
frame_generator = run_player_detection(source_video_path="../test_videos/test_video2.mp4")

video_info = sv.VideoInfo.from_video_path("../test_videos/test_video2.mp4")
with sv.VideoSink("result", video_info) as sink:
    for frame in frame_generator:
        sink.write_frame(frame)

        cv2.imshow("frame", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
        
    cv2.destroyAllWindows()


[ WARN:0@1201.630] global cap.cpp:643 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.10.0) /io/opencv/modules/videoio/src/cap_images.cpp:430: error: (-215:Assertion failed) !filename_pattern.empty() in function 'open'


